In [2]:
import cv2
import numpy as np

In [3]:
class USB_Camera(object): 
    def __init__(self):
        self.cam_matrix_left = np.array([[1138.9,0,185.5675],[0.,1128.5,206.0001],[0.,0.,1.]])
        self.distortion_l = np.array([[0.2616,-1.3541,4.4189,-0.0095,-0.0201]])
        self.cam_matrix_right = np.array([[669.6477,0,333.6960],[0.,666.8842,247.8914],[0.,0.,1.]])
        self.distortion_r = np.array([[-0.3811,1.3331,-15.3512,-0.00023421,0.00020229]])
        self.R = np.array([[0.999,0.0153,0.0064],[-0.0152,0.9997,-0.0205],[-0.0067,0.0204,0.9998]])
        self.T = np.array([[-78.1370],[2.4380],[-0.7273]])
        self.focal_length = 6.00
        self.baseline = np.abs(self.T[0])

def triangulation(pointl_vec,pointr_vec,R,t,cam_matrix_left,cam_matrix_right,n):
    pointl_cam_vec = []
    pointr_cam_vec = []
    for pointl in pointl_vec:
        pointl_cam_vec.append([(pointl[0] - cam_matrix_left[0, 2]) / cam_matrix_left[0, 0],(pointl[1] - cam_matrix_left[1, 2]) / cam_matrix_left[1, 1]])
    for pointr in pointr_vec:
        pointr_cam_vec.append([(pointr[0] - cam_matrix_right[0, 2]) / cam_matrix_right[0, 0],(pointr[1] - cam_matrix_right[1, 2]) / cam_matrix_right[1, 1]])
    T1 = np.array([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,1.,0.]])
    T2 = np.concatenate((R,t),axis=1)
    # print(T1,T2)
    pointl_cam_vec = np.array(pointl_cam_vec).transpose()
    pointr_cam_vec = np.array(pointr_cam_vec).transpose()
    # print(pointl_cam_vec)
    # print(pointr_cam_vec)

    pts_4d = np.zeros((4,n))
    cv2.triangulatePoints(T1,T2,pointl_cam_vec,pointr_cam_vec,pts_4d)
    pts_3d = []
    for i in range(n):
        x = pts_4d[0,i]/pts_4d[3,i]
        y = pts_4d[1,i]/pts_4d[3,i]
        z = pts_4d[2,i]/pts_4d[3,i]
        pts_3d.append([x,y,z])
    pts_3d = np.array(pts_3d)

    return pts_3d

In [8]:
# 创建两个摄像头对象
cap_left = cv2.VideoCapture(1)
cap_right = cv2.VideoCapture(0)

# 加载人脸检测器
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#加载相机参数
config = USB_Camera()

while True:
    # 从左摄像头读取视频帧
    ret_left, frame_left = cap_left.read()
    
    # 从右摄像头读取视频帧
    ret_right, frame_right = cap_right.read()

    if not ret_left or not ret_right:
        break
    
    # 将视频帧调整为相同大小
    frame_left = cv2.resize(frame_left, (640, 480))
    frame_right = cv2.resize(frame_right, (640, 480))

    # 将视频帧转换为灰度图像
    gray_left = cv2.cvtColor(frame_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(frame_right, cv2.COLOR_BGR2GRAY)

    # 检测人脸
    faces_left = face_cascade.detectMultiScale(gray_left, 1.3, 5)
    faces_right = face_cascade.detectMultiScale(gray_right, 1.3, 5)

    # 遍历左摄像头中检测到的人脸（因为可能检测到多个人脸）
    for (xl, yl, wl, hl),(xr, yr, wr, hr) in zip(faces_left,faces_right):
        #print((xl, yl, wl, hl),(xr, yr, wr, hr))
        # 在左摄像头视频帧上绘制人脸矩形
        cv2.rectangle(frame_left, (xl, yl), (xl+wl, yl+hl), (255, 0, 0), 2)
        cv2.rectangle(frame_right, (xr, yr), (xr+wr, yr+hr), (255, 0, 0), 2)
        # 计算人脸到左右相机之间的距离（示例中仅为演示，距离计算方法可能有所不同）
        positionl=[[xl+hl/2,yl+wl/2]]
        positionr=[[xr+hr/2,yr+wr/2]]
        #print('positionl,positionr',positionl,positionr)
        pts_3d=triangulation(positionl,positionr,config.R,config.T,config.cam_matrix_left,config.cam_matrix_right,1)
        #print('pts_3d',pts_3d)
        distance =np.linalg.norm(pts_3d, axis=1, keepdims=True)[0][0]/1000
        #print('distance',distance)
        
        # 在视频帧上显示距离信息
        cv2.putText(frame_right, f'Distance: {distance:.3f}m', (xr, yr-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        cv2.putText(frame_left, f'Distance: {distance:.3f}m', (xl, yl-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # 显示左右两个摄像头的视频帧
    cv2.imshow('Left Camera', frame_left)
    cv2.imshow('Right Camera', frame_right)

    # 按下 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(1) & 0xFF == ord('Q'):
        break

# 释放摄像头并关闭窗口
cap_left.release()
cap_right.release()
cv2.destroyAllWindows()